In [1]:
#CELDA 1: Configuraciones básicas

import re
import pandas as pd
from pathlib import Path
from IPython.display import display

%run 00_setup.ipynb

#RUTAS
from src.config.rutas import RUTA_DATASET
   

In [2]:
#CELDA 2: Carga del dataset

dt = pd.read_csv(RUTA_DATASET)

print("Num. de filas y columnas:")
print(dt.shape)         #Muestra la forma del dataset por filas y columnas
dt.head()

Num. de filas y columnas:
(16127, 5)


,texto,label,vector,fuente,id
0,Moby Pub Quiz.Win a £100 High Street prize if ...,1,sms,mendeley,0
1,https://www.documenters.org,0,url,phiusiil,1
2,scan system adware wrongful 6130320948821283 c...,1,email,kaggle_email,2
3,mit student thesis writeups hi vince krishna t...,0,email,kaggle_email,3
4,https://swn9klhal.web.app/,1,url,phiusiil,4


In [3]:
#CELDA 3: Checklist previo (se realiza de nuevo antes de la limpieza)

print("Valores nulos por columnas:")
display(dt.isna().sum())

print("\nNum. total de filas duplicadas:")
display(dt.duplicated().sum())

print("Tipo de datos por columnas:")
display(dt.dtypes)

print("\nFilas por vector:")
display(dt["vector"].value_counts())

print("\nFilas por clase:")
display(dt["label"].value_counts())

Valores nulos por columnas:


texto     0
label     0
vector    0
fuente    0
id        0
dtype: int64


Num. total de filas duplicadas:


np.int64(0)

Tipo de datos por columnas:


texto     object
label      int64
vector    object
fuente    object
id         int64
dtype: object


Filas por vector:


vector
url      6000
email    6000
sms      4127
Name: count, dtype: int64


Filas por clase:


label
0    9000
1    7127
Name: count, dtype: int64

In [4]:
#CELDA 4: Función de limpieza de texto

def limpiar_texto(t: str) -> str:

    #Se realiza una limpieza ligera y segura para los tres tipos de vecrores principales preservando la semantica

    if not isinstance(t, str):      #Se comprueba si la variable pasada es una cadena
        t = str(t)
    
    t = t.lower()                           #Convertir a minusculas
    t = re.sub(r"\s", " ", t)               #Elimina saltos de linea
    t = re.sub(r"[^\x20-\x7E]", "", t)      #Elimina carcteres no imprimibles
    t = t.strip()                           #Compacta los espacios

    return t

In [5]:
#CELDA 5: Aplicar la funcion de limpieza

#Se aplica la funcion creada en la anterior celda
dt["texto_preprocesado"] = dt["texto"].apply(limpiar_texto)

print("Conversion:")
display(dt[["texto", "texto_preprocesado"]].head())

#Comprobacion post-limpieza

print("\nTextos vacios post-limpieza:")
display((dt["texto_preprocesado"].str.len() == 0).sum())

print("\nEjemplos aleatorios post-limpieza:")
display(dt.sample(5)[["texto", "texto_preprocesado"]])

#Comprobamso si el proceso de limpieza altera mucho la semantica de las urls
print("\nURLs post-limpieza:")
display(dt[dt["vector"] == "url"].sample(5)[["texto", "texto_preprocesado"]])

Conversion:


,texto,texto_preprocesado
0,Moby Pub Quiz.Win a £100 High Street prize if ...,moby pub quiz.win a 100 high street prize if u...
1,https://www.documenters.org,https://www.documenters.org
2,scan system adware wrongful 6130320948821283 c...,scan system adware wrongful 6130320948821283 c...
3,mit student thesis writeups hi vince krishna t...,mit student thesis writeups hi vince krishna t...
4,https://swn9klhal.web.app/,https://swn9klhal.web.app/



Textos vacios post-limpieza:


np.int64(1)


Ejemplos aleatorios post-limpieza:


,texto,texto_preprocesado
11539,Badrith is only for chennai:)i will surely pic...,badrith is only for chennai:)i will surely pic...
6942,https://saika69sex.monster/qdoj8,https://saika69sex.monster/qdoj8
15870,\tMessage Important information for O2 user. T...,message important information for o2 user. tod...
5832,https://www.corp.at,https://www.corp.at
14052,https://www.biografiadechile.cl,https://www.biografiadechile.cl



URLs post-limpieza:


,texto,texto_preprocesado
3663,http://www.angel2gether.de,http://www.angel2gether.de
11178,https://espkt.info/de/home/login-online-bankin...,https://espkt.info/de/home/login-online-bankin...
9401,https://www.naturescapes.net,https://www.naturescapes.net
9310,https://www.windmillhillcityfarm.org.uk,https://www.windmillhillcityfarm.org.uk
3321,http://www.medresearchgroup.com,http://www.medresearchgroup.com


In [6]:
#CELDA 6: Eliminar textos vacios tras el preprocesamiento

num_vacios = (dt["texto_preprocesado"].str.len() == 0).sum()
print("Textos vacios: ", num_vacios)

dt = dt[dt["texto_preprocesado"].str.len() > 0].reset_index(drop=True)      #Se reinicia los indices en el DataFrame

print("Dataset final tras eliminar vacios:", dt.shape)

Textos vacios:  1
Dataset final tras eliminar vacios: (16126, 6)
